<a href="https://colab.research.google.com/github/pachterlab/BI-BE-CS-183-2023/blob/main/HW8/Problem3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Bi/Be/Cs 183 2022-2023: Intro to Computational Biology
TAs: Meichen Fang, Tara Chari, Zitong (Jerry) Wang

**Submit your notebooks by sharing a clickable link with Viewer access. Link must be accessible from submitted assignment document.**

Make sure Runtime $\rightarrow$ Restart and run all works without error

# HW 8 Problem 3

Chimpanzees are the closest living relatives to humans. Here we will be looking at differences between the two genomes. 

In this problem you will implement the Needleman-Wunsch algorithm for global alignment between a pair of sequences of human and chimpanzee FOXP2 genes [1]. You will be reading in a FASTA file of DNA sequences and run the algorithm to find their optimal alignment given a set of rewards and penalties for the possible nucleotide pairs. This algorithm allows for gaps in the alignment, and constructs the global optimal alignment by iteratively finding the solutions for sets of small subsequences and their alignments.

FOXP2 is a gene that has been implicated in speech and language development in humans, where mutations in the gene can inhibit speech. Interestingly, FOXP2s sequence and function are different between chimpanzees and humans, and its mechanism(s) is an area of study for researchers in understanding how we process and develop language.

[1] Enard W, Przeworski M, Fisher SE, Lai CS, Wiebe V, Kitano T, Monaco AP, Pääbo S. Molecular evolution of FOXP2, a gene involved in speech and language. Nature. 2002 Aug 22;418(6900):869-72. doi: 10.1038/nature01025. Epub 2002 Aug 14. PMID: 12192408.



##**Import data and install packages**

In [1]:
import numpy as np
import scipy.io as sio
import pandas as pd
import matplotlib.pyplot as plt #Can use other plotting packages like seaborn

In [2]:
#Download the FASTA file with a pair of nucleotide sequences

!wget --content-disposition 'https://drive.google.com/uc?export=download&id=1GFYciWTl2Objrplb1XqNa0pWzTBvXbsm'


--2023-02-22 23:30:13--  https://drive.google.com/uc?export=download&id=1GFYciWTl2Objrplb1XqNa0pWzTBvXbsm
Resolving drive.google.com (drive.google.com)... 108.177.112.139, 108.177.112.102, 108.177.112.113, ...
Connecting to drive.google.com (drive.google.com)|108.177.112.139|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-0k-3c-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/1d4esi84mt7264lbhjpbjp723o0si2k5/1677108600000/14515314329444727017/*/1GFYciWTl2Objrplb1XqNa0pWzTBvXbsm?e=download&uuid=4079cb17-8a1a-4326-9f61-6e84ab14e7e9 [following]
--2023-02-22 23:30:14--  https://doc-0k-3c-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/1d4esi84mt7264lbhjpbjp723o0si2k5/1677108600000/14515314329444727017/*/1GFYciWTl2Objrplb1XqNa0pWzTBvXbsm?e=download&uuid=4079cb17-8a1a-4326-9f61-6e84ab14e7e9
Resolving doc-0k-3c-docs.googleusercontent.com (doc-0k-3c-docs.googleusercontent.com)... 173.194.197.132, 

## **Read in data for analysis**

**The data**

Here we print the contents of the FASTA file of a pair of DNA sequences. FASTA is a common text file format in which nucleotide sequences are stored. Commonly, the name of the sequence follows a '>'. The following line then has the sequence itself, terminated by an '*'.

In [ ]:
#Print the FASTA file we downloaded
!cat seqs.fasta

>sequence1
TGCATCAGCC*
>sequence2
TGCGTCAGTCAGCC*


## **Problem 3 (40 points)**

In the Needleman-Wunsch (NW) algorithm we begin by constructing a score matrix $\mathbf{F}$, an $(m+1)\times(n+1)$ matrix for two sequences $x,y$ of length $m$ and $n$ respectively. Each entry $i,j$ corresponds to the alignment score of the nucleotides in at $x_i \text{ and } y_j$.

See the example (blank) score matrix for two sequences $x$ = 'GATTACA', $y$ = 'GCATGCT'.

Nucleotide pairs can either be aligned to each other (e.g. $F(1,1)$ denotes 'G' mapped to 'G') or to a gap (e.g. $F(1,0)$ denotes 'G' mapped to '-'). Pairs can also be aligned if they do not match (e.g. $F(1,2)$ denotes 'G' mapped to 'A'). 







<center><img src="https://drive.google.com/uc?export=view&id=1Qfzs96bRkU2YJwx8Ss9VcjRN2EPUyLY4" alt="exGrid" width="300" height="300"><center>

To calculate a score/entry for $F_{i,j}$, we will use the scores of $F_{i-1,j-1},F_{i-1,j}$ and $F_{i,j-1}$. The score for $F_{i,j}$ is defined as the maximum score of (1) $x_i$ being aligned to $y_j$, (2) $x_i$ being aligned to a gap '-', and (3) $y_j$ being aligned to a gap '-'. Thus the score is calculated as:

\begin{align}
F_{i,j} = max\begin{cases} 
      F_{i-1,j-1} + s(x_i,y_j) & (1) \\
      F_{i-1,j} - d & (2) \\
      F_{i,j-1} - d & (3)
   \end{cases}
\end{align}

$s()$ denotes a score function, which we will define below, that returns a reward for two matched nucleotides or a penalty for mismatched bases.
We will also set a value for the 'gap penalty' $d$.

We can draw this score calculation as:

<center><img src="https://drive.google.com/uc?export=view&id=1MZf_hZ_7Tprj0hV0rpwsBWXBmrmFvWt_" alt="exScore" width="300" height="160"><center>

### **a) Read in the DNA sequences from the FASTA file. Extract just the sequence strings, *not including* the asterisk at the end, and print the two strings. (5 points)** 

You may want to save the two strings as two variables or as a list.

### **b) Fill in the boundary conditions of $\mathbf{F}$. (5 points)**
The first column and row denote all bases in that column or row being mapped to gaps '-'. Assuming $F_{0,0} = 0$, fill in row 0 and column 0 accordingly, using a **gap penalty $\mathbf{d = -2}$**. 

**Print your full $\mathbf{F}$ matrix after initialization.**

### **c) Implement the s() (score) function. (5 points)**

Fill in the s() function below to take in two nucleotide bases and output a **reward of 2** if the bases match, and a **penalty of -1** for mismatched bases.

In [ ]:
def s():

### **d) Fill in the rest of the $\mathbf{F}$ matrix, row by row. (5 points)**

Fill in the $\mathbf{F}$ matrix using the score function described in the main Problem statement, using the s() function from **c)** and a **gap penalty $\mathbf{d = -2}$**.

**Print the full final $\mathbf{F}$ matrix.**

### **e) Beginning at bottom corner, traceback the alignment and output the final alignment of the two sequences (10 points)**

At each step in the traceback we look at the entries that $F_{i,j}$ was derived from, $F_{i-1,j-1}, F_{i-1,j},F_{i,j-1}$. We take a 'step' back to the previous entry that has the max score, and add a pair of symbols/nucleotides to the alignment accordingly.

If the step is to (1) $F_{i-1,j-1}$ (diagonal arrow), $x_i$ and $y_j$ are added to the alignment. If the step is to (2) $F_{i-1,j}$ (vertical arrow), $x_i$ and '-' (gap character) are added, or '-' and $y_j$ if (3) $F_{i,j-1}$ (horizontal arrow) is chosen. See arrow diagram below in example traceback.


If there is a tie for the max entry, choose any of the entries in that tie with equal probability. 

See an example traceback below, where the red arrows denote the optimal alignment between sequences x: ACGCATCA and y: ACTGATTCA which is:

A C T G - A T T C A

A C - G C A T - C A


Example: Starting at the right corner, we first select the cell with 6 (the i-1, j-1 cell) as it has the max score out of (4,6,4). Thus we choose to align A from x and A from y. The next step selects the cell with 4 (another i-1, j-1 cell), and thus C and C are aligned. In the next step the cell with 6 is selected (i,j-1) thus the T from y is aligned to a gap '-'.

<center><img src="https://drive.google.com/uc?export=view&id=17M9wpDRCdDnXKvgSFL7S_nw7ZeIaKC0E" alt="exScore" width="450" height="350"><center>

### **f) Change the score function to adapt the mismatch penalties to account for more or less likely nucleotide substitutions. (5 points)**

Transitions within the purine class (A <--> G) or within the pyrimidine class (C <--> T) are more likely and thus have a **penalty of -1**. Transversions, across classes (e.g. A --> T) have a **penalty of -2** as they are less likely.

**Define a new s() function to implement these penalty changes.**

In [ ]:
def s():

### **g) Re-run the NW algorithm with new penalties, report your new final alignment with the penalty changes/new score function in f), and print the full (final) $\mathbf{F}$ matrix. (5 points)**

Use the adapted mismatch penalties in part f) and **set gap penalty d = -4**. This will ensure you a unique alignment solution